In [7]:
import keras

import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import csv

2024-04-07 22:28:56.122969: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 22:28:56.123142: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 22:28:56.125733: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 22:28:56.158481: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 22:28:56.738819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

Cargamso el dataset:

In [8]:
iris = datasets.load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

Creamos el dataset en pandas y lo sufleamos

In [9]:
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                        columns= iris['feature_names'] + ['target'])
print("Numero de elementos de cada clase")
print(iris_df['target'].value_counts())

Numero de elementos de cada clase
target
0.0    50
1.0    50
2.0    50
Name: count, dtype: int64


In [4]:
def normalize_l2(df, columns):
  """
  Normalizes specified columns in a dataframe using L2 norm.

  Args:
      df: The pandas dataframe.
      columns: List of column names to normalize.

  Returns:
      A new dataframe with normalized columns.
  """
  df_norm = df.copy()
  for col in columns:
    # Calculate L2 norm for each row
    norm = df[col].apply(lambda x: np.sqrt(sum(x**2 for x in df[col])))

    # Avoid division by zero (rows with all zeros)
    norm[norm == 0] = 1

    # Normalize the column values
    df_norm[col] = df[col] / norm
  return df_norm

In [10]:
iris_shuffleados = iris_df.sample(frac=1, random_state=0)

tamañoIris = len(iris_shuffleados)
tamañoResto = int(tamañoIris * 0.8)


iris_trainAll = iris_shuffleados.iloc[:tamañoResto]
iris_pruebaAll = iris_shuffleados.iloc[tamañoResto:]


tamañoEntrenamiento = int(tamañoResto * 0.8)

irisEntrenamientoAll = iris_trainAll.iloc[:tamañoEntrenamiento]
irisValidacionAll = iris_trainAll.iloc[tamañoEntrenamiento:]

print(len(iris_trainAll), len(iris_pruebaAll))
print(len(irisEntrenamientoAll), len(irisValidacionAll))

120 30
96 24


In [11]:
irisEntrenamientoAll = irisEntrenamientoAll.to_numpy()
x = irisEntrenamientoAll[:, :-1]
y = irisEntrenamientoAll[:, -1]

irisValidacionAll = irisValidacionAll.to_numpy()
x_val = irisValidacionAll[:, :-1]
y_val = irisValidacionAll[:, -1]


iris_pruebaAll = iris_pruebaAll.to_numpy()
x_test = iris_pruebaAll[:, :-1]
y_test = iris_pruebaAll[:, -1]

In [12]:
y

array([2., 1., 0., 2., 0., 2., 0., 1., 1., 1., 2., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 2., 1., 0., 0., 2., 0., 0., 1., 1., 0., 2., 1., 0., 2.,
       2., 1., 0., 1., 1., 1., 2., 0., 2., 0., 0., 1., 2., 2., 2., 2., 1.,
       2., 1., 1., 2., 2., 2., 2., 1., 2., 1., 0., 2., 1., 1., 1., 1., 2.,
       0., 0., 2., 1., 0., 0., 1., 0., 2., 1., 0., 1., 2., 1., 0., 2., 2.,
       2., 2., 0., 0., 2., 2., 0., 2., 0., 2., 2.])

In [20]:

my_callbacks = [
    keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint(filepath='modelo.{epoch:02d}-{val_loss:.2f}.keras'),
    keras.callbacks.TensorBoard(log_dir='./logs'),
]

neuronas = [12, 30, 48]
learning_rates = [0.01, 0.05, 0.1]
epochs = [100, 200, 300]
momentums = [0.1, 0.25, 0.5]
capas = [5, 7, 10]

for neurona in neuronas:
    for i in [10, 15 , 20]:
       for lr in learning_rates:
            for epoch in epochs:
                for momentum in momentums:
                    modelo = keras.Sequential()
                    modelo.add(keras.layers.Dense(units=16, activation="relu", input_shape=(4,)))
                    for j in range(i):
                        modelo.add(keras.layers.Dense(neurona, activation="relu"))
                        if j == i -1:
                            print("-" * 100 + "\n")
                            print("modelo con " + str(neurona) +" neuronas por capa, " + str(i) +" capas")
                            modelo.add(keras.layers.Dense(3, activation="sigmoid"))
                            
                            nombre = 'str(neurona) neuronas  + i +  capas'
                            modelo.name = str(neurona)
                            
                            optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
                            modelo.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
                            modelo.fit(x=x, y=y, epochs=epoch, batch_size=32, validation_data=(x_val, y_val))
                            
                            loss, accuracy = modelo.evaluate(x_test, y_test)
                            print("Validation Loss:", loss)
                            print("Validation Accuracy:", accuracy)
                            with open("data.csv", "a", newline="") as csvfile:
                                writer = csv.writer(csvfile)
                                #writer.writerow(["Capas", "Neuronas por capa", "loss", "accuracy"])
                                writer.writerow([i, neurona, lr, epoch, momentum ,loss, accuracy])




----------------------------------------------------------------------------------------------------

modelo con 12 neuronas por capa, 10 capas
Epoch 1/100


/home/david/Trabajos/IA2/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.3815 - loss: 1.0956 - val_accuracy: 0.2500 - val_loss: 1.0995
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3776 - loss: 1.0957 - val_accuracy: 0.2500 - val_loss: 1.0999
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3464 - loss: 1.0961 - val_accuracy: 0.2500 - val_loss: 1.1003
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3503 - loss: 1.0957 - val_accuracy: 0.2500 - val_loss: 1.1007
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3424 - loss: 1.0963 - val_accuracy: 0.2500 - val_loss: 1.1011
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3776 - loss: 1.0951 - val_accuracy: 0.2500 - val_loss: 1.1016
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3776 - loss: 1.0948 - val_accuracy: 0.2500 - val_loss: 1.1020
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3503 - loss: 1.0947 - val_accuracy: 0.2500 - val_loss: 1.1022
Epoc

2024-04-06 22:45:46.579746: W tensorflow/core/data/root_dataset.cc:350] Optimization loop failed: CANCELLED: Operation was cancelled


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7018 - loss: 0.8393 - val_accuracy: 1.0000 - val_loss: 0.0771
Epoch 275/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9740 - loss: 0.1326 - val_accuracy: 1.0000 - val_loss: 0.0692
Epoch 276/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9818 - loss: 0.0982 - val_accuracy: 1.0000 - val_loss: 0.0463
Epoch 277/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9740 - loss: 0.0872 - val_accuracy: 1.0000 - val_loss: 0.0360
Epoch 278/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9688 - loss: 0.1060 - val_accuracy: 1.0000 - val_loss: 0.0405
Epoch 279/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9740 - loss: 0.1003 - val_accuracy: 1.0000 - val_loss: 0.0499
Epoch 280/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9805 - loss: 0.0819 - val_accuracy: 0.8750 - val_loss: 0.4338
Epoch 281/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7760 - loss: 0.6420 - val_accuracy: 0.7500 - val_los

In [16]:
import pandas as pd
data = pd.read_csv("data(1).csv")

def weighted_score(row):
    loss_weight = 0.7  # Adjust weights based on your problem's priorities
    accuracy_weight = 1 - loss_weight
    return (loss_weight * (1 - row["Loss"])) + (accuracy_weight * row["accuracy"])

# Add a column for the weighted score
data["weighted_score"] = data.apply(weighted_score, axis=1)

# Identify the row with the highest weighted score
best_row = data.nlargest(1, "weighted_score")

# Print the best hyperparameters and corresponding metrics
print("Best Hyperparameters:")
print("Capas:", best_row.iloc[0]["Capas"])
print("Neuronas por capa:", best_row.iloc[0]["Neuronas por capa"])
print("Learning rate:", best_row.iloc[0]["Learning rate"])
print("Epochs:", best_row.iloc[0]["Epochs"])
print("Momentum:", best_row.iloc[0]["Momentum"])
print("Loss:", best_row.iloc[0]["Loss"])
print("Accuracy:", best_row.iloc[0]["accuracy"])
print("Weighted Score:", best_row.iloc[0]["weighted_score"])


Best Hyperparameters:
Capas: 6.0
Neuronas por capa: 48.0
Learning rate: 0.05
Epochs: 300.0
Momentum: 0.25
Loss: 0.0180139038711786
Accuracy: 1.0
Weighted Score: 0.987390267290175


In [33]:

# Define the model
model = keras.Sequential([
  keras.layers.Dense(units=16, activation="relu", input_shape=(4,)),
  keras.layers.Dense(units=30, activation="relu"),
  keras.layers.Dense(units=48, activation="relu"),
  keras.layers.Dense(units=48, activation="relu"),
  keras.layers.Dense(units=48, activation="relu"),
  keras.layers.Dense(units=48, activation="relu"),
  keras.layers.Dense(3, activation="sigmoid")
])

optimizer = keras.optimizers.SGD(learning_rate=0.05, momentum=0.25)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x=x, y=y, epochs=300, batch_size=32, validation_data=(x_val, y_val))

loss, accuracy = model.evaluate(x_test, y_test)


Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.3581 - loss: 1.1115 - val_accuracy: 0.2500 - val_loss: 1.0527
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4766 - loss: 1.0290 - val_accuracy: 0.5833 - val_loss: 1.0152
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5846 - loss: 0.9795 - val_accuracy: 0.7500 - val_loss: 0.9367
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6732 - loss: 0.9186 - val_accuracy: 0.7500 - val_loss: 0.8269
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6654 - loss: 0.8268 - val_accuracy: 0.7500 - val_loss: 0.6888
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7318 - loss: 0.7220 - val_accuracy: 0.9167 - val_loss: 0.5517
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7448 - loss: 0.6715 - val_accuracy: 0.9583 - val_loss: 0.4768
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8464 - loss: 0.5797 - val_accuracy: 0.9583 - val_loss: 0.37

In [34]:
loss, accuracy

(0.04059350863099098, 1.0)